<a href="https://colab.research.google.com/github/Srikanth1234567808/inter_IIT_trails-/blob/main/Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

csv_file = "/content/ticker_lisst.csv"
ticker_symbols = pd.read_csv(csv_file)['Tickers'].tolist()

data = yf.download(ticker_symbols, start="2023-09-30", end="2024-10-01", progress=False)

ohlc_data = data[['Open', 'High', 'Low', 'Close']]
ohlc_data.columns = ['_'.join(col) for col in ohlc_data.columns]

def calculate_pct_change(ticker):
    high_col = f'High_{ticker}'
    close_col = f'Close_{ticker}'
    return ((ohlc_data[high_col] - ohlc_data[close_col].shift(1)) / ohlc_data[close_col].shift(1)) * 100

pct_changes = {ticker: calculate_pct_change(ticker) for ticker in ticker_symbols}
pct_change_df = pd.DataFrame(pct_changes)

ohlc_data = pd.concat([ohlc_data, pct_change_df.add_suffix('_Pct_Change_High_Close')], axis=1)

def percentile_exc(data, q):
    data_sorted = np.sort(data.dropna())
    n = len(data_sorted)

    if n == 0:
        return np.nan

    rank = q * (n + 1) - 1
    if rank < 0:
        return data_sorted[0]
    elif rank >= n - 1:
        return data_sorted[-1]
    else:
        lower = int(np.floor(rank))
        upper = int(np.ceil(rank))
        return data_sorted[lower] + (rank - lower) * (data_sorted[upper] - data_sorted[lower])

percentile_10_exc = pct_change_df.apply(lambda x: percentile_exc(x, 0.10))

adjusted_percentile_10_exc = percentile_10_exc - 0.01

print("The adjusted value where 90% of the percentage changes are above for each company (after subtracting 0.01):")
print(adjusted_percentile_10_exc)
